In [1]:
from pynq import Overlay, allocate, PL
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

PL.reset()
overlay = Overlay('design_1.bit')

In [2]:
print('IP blocks :', list(overlay.ip_dict.keys()))

IP blocks : ['yuv_filter_0', 'axi_dma_0', 'processing_system7_0']


In [3]:
yuv_filter     = overlay.yuv_filter_0
#dma_0 = overlay.axi_dma_0

dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

CONTROL_REGISTER = 0x0

def get_register_offset(overlay, ip, parameter):
    return overlay.ip_dict[ip]['registers'][parameter]['address_offset']

SCALE_REGISTER = get_register_offset(overlay, 'yuv_filter_0', 'scale_Y')

In [4]:
def yuv_filter_fpga(image, scale_Y):
    
    shape = image.shape
    
    yuv_filter.write(SCALE_REGISTER, scale_Y)
    yuv_filter.write(CONTROL_REGISTER, 0x01)
    
    input_buffer = allocate(shape=(shape[0], shape[1], 4), dtype=np.uint8)
    output_buffer = allocate(shape=(shape[0], shape[1], 4), dtype=np.uint8)
    
    input_buffer[:,:,0:3] = image
    
    dma_send.transfer(input_buffer)
    dma_recv.transfer(output_buffer)
    dma_send.wait()
    dma_recv.wait()
    
    return output_buffer[:,:,0:3]

In [ ]:
%matplotlib inline

import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image = mpimg.imread('input.jpg')

# If the image has an alpha channel, remove it
if image.shape[2] == 4:
    image = image[:, :, :3]

# If the image is in float format (0-1), convert it to uint8 (0-255)
if image.dtype == np.float32 or image.dtype == np.float64:
    image = (image * 255).astype(np.uint8)

start_time = time.time()
filtered_image = yuv_filter_fpga(image, 1)
processing_time = time.time() - start_time

plt.imshow(filtered_image)
plt.title(f"Processing time {processing_time}")
plt.show()